In [1]:
import pandas as pd
import altair as alt

In [2]:
start_date= '2017-01-01'

## Vergleich Marktpreis und effektive Vergütung über die Zeit

Wann lagen die Vergütungen über dem Marktpreis und wann darunter?

In [3]:
d1 = pd.read_csv("calc/marktpreis_quarter.csv").query("Datum >= @start_date")
d1 = d1[['Datum', 'chf_kwh_0', 'chf_kwh_4_6', 'chf_kwh_8']]
d1['year'] = pd.to_datetime(d1['Datum']).dt.year
d1

,Datum,chf_kwh_0,chf_kwh_4_6,chf_kwh_8,year
8,2017-01-01,5.643,5.643,8.000,2017
9,2017-04-01,3.557,4.600,8.000,2017
10,2017-07-01,3.897,4.600,8.000,2017
11,2017-10-01,6.958,6.958,8.000,2017
12,2018-01-01,6.175,6.175,8.000,2018
13,2018-04-01,4.309,4.600,8.000,2018
14,2018-07-01,6.508,6.508,8.000,2018
15,2018-10-01,8.033,8.033,8.033,2018
16,2019-01-01,4.897,4.897,8.000,2019
17,2019-04-01,3.805,4.600,8.000,2019


Marktpreis:

In [4]:
markt = alt.Chart(d1).mark_line(interpolate='step-after').encode(
    alt.X('Datum:T'),
    alt.Y('chf_kwh_4_6:Q'),
    color=alt.value("#ae49a2")
)
markt

alt.Chart(...)

Nach Einwohner gewichteter, mittlere effektive Vergütung pro Jahr:

In [5]:
d3 = pd.read_csv("calc/preis_gemeinde_jahr.csv")
d3['Datum'] = pd.to_datetime(d3.year, format='%y')
d3['year'] = pd.to_datetime(d3.Datum).dt.year
d3

,ID,year,strompreis,Gemeinde,Einwohner,Datum
0,1,2017,6.23,Aeugst am Albis,1981,2017-01-01
1,1,2018,5.53,Aeugst am Albis,1981,2018-01-01
2,1,2019,5.53,Aeugst am Albis,1981,2019-01-01
3,1,2020,7.29,Aeugst am Albis,1981,2020-01-01
4,1,2021,7.30,Aeugst am Albis,1981,2021-01-01
...,...,...,...,...,...,...
14217,6810,2019,9.50,La Baroche,1142,2019-01-01
14218,6810,2020,7.68,La Baroche,1142,2020-01-01
14219,6810,2021,14.03,La Baroche,1142,2021-01-01
14220,6810,2022,30.00,La Baroche,1142,2022-01-01


In [6]:
# Gewichteter Mittelwert nach Jahr. Gewichtung nach Anzahl Einwohner
total_einwohner = d3.groupby('ID').Einwohner.first().sum()
total_einwohner



8523044

In [7]:
d4 = d3.copy()
d4['weight'] = d4.Einwohner / total_einwohner
d4 = d4.groupby('year').apply(lambda x: (x['strompreis'] * x['weight']).sum()).reset_index().rename({0: 'effektive'}, axis=1)
d4

,year,effektive
0,2017,8.702552
1,2018,8.120404
2,2019,8.618476
3,2020,8.946350
4,2021,9.594124
5,2022,12.741219
6,2023,14.653950


In [8]:
diff = pd.merge(d1, d4[['year', 'effektive']], on='year', how='left')
diff

,Datum,chf_kwh_0,chf_kwh_4_6,chf_kwh_8,year,effektive
0,2017-01-01,5.643,5.643,8.000,2017,8.702552
1,2017-04-01,3.557,4.600,8.000,2017,8.702552
2,2017-07-01,3.897,4.600,8.000,2017,8.702552
3,2017-10-01,6.958,6.958,8.000,2017,8.702552
4,2018-01-01,6.175,6.175,8.000,2018,8.120404
5,2018-04-01,4.309,4.600,8.000,2018,8.120404
6,2018-07-01,6.508,6.508,8.000,2018,8.120404
7,2018-10-01,8.033,8.033,8.033,2018,8.120404
8,2019-01-01,4.897,4.897,8.000,2019,8.618476
9,2019-04-01,3.805,4.600,8.000,2019,8.618476


In [9]:

diff = diff.query('Datum >= "'+start_date+'"')
diff.loc[diff.chf_kwh_4_6 >= diff.effektive, 'winner'] = 'Marktpreis'
diff.loc[diff.chf_kwh_4_6 >= diff.effektive, 'lower'] = diff.effektive
diff.loc[diff.chf_kwh_4_6 >= diff.effektive, 'upper'] = diff.chf_kwh_4_6
diff.loc[diff.chf_kwh_4_6 < diff.effektive, 'winner'] = 'Versorger'
diff.loc[diff.chf_kwh_4_6 < diff.effektive, 'lower'] = diff.chf_kwh_4_6
diff.loc[diff.chf_kwh_4_6 < diff.effektive, 'upper'] = diff.effektive

diff['Date_start'] = diff['Datum']
diff['Date_end'] = diff['Datum'].shift(-1)

diff.to_excel('diff.xlsx', index=False)
diff

,Datum,chf_kwh_0,chf_kwh_4_6,chf_kwh_8,year,effektive,winner,lower,upper,Date_start,Date_end
0,2017-01-01,5.643,5.643,8.000,2017,8.702552,Versorger,5.643000,8.702552,2017-01-01,2017-04-01
1,2017-04-01,3.557,4.600,8.000,2017,8.702552,Versorger,4.600000,8.702552,2017-04-01,2017-07-01
2,2017-07-01,3.897,4.600,8.000,2017,8.702552,Versorger,4.600000,8.702552,2017-07-01,2017-10-01
3,2017-10-01,6.958,6.958,8.000,2017,8.702552,Versorger,6.958000,8.702552,2017-10-01,2018-01-01
4,2018-01-01,6.175,6.175,8.000,2018,8.120404,Versorger,6.175000,8.120404,2018-01-01,2018-04-01
5,2018-04-01,4.309,4.600,8.000,2018,8.120404,Versorger,4.600000,8.120404,2018-04-01,2018-07-01
6,2018-07-01,6.508,6.508,8.000,2018,8.120404,Versorger,6.508000,8.120404,2018-07-01,2018-10-01
7,2018-10-01,8.033,8.033,8.033,2018,8.120404,Versorger,8.033000,8.120404,2018-10-01,2019-01-01
8,2019-01-01,4.897,4.897,8.000,2019,8.618476,Versorger,4.897000,8.618476,2019-01-01,2019-04-01
9,2019-04-01,3.805,4.600,8.000,2019,8.618476,Versorger,4.600000,8.618476,2019-04-01,2019-07-01


In [10]:
diff_chart = alt.Chart(diff).mark_rect(clip=True).encode(
    alt.Y('lower:Q'),
    alt.Y2('upper:Q'),
    alt.X('Date_start:T').scale(domain=["2017-01-01", "2023-12-31"]),
    alt.X2('Date_end:T'),
    alt.Color('winner:N').scale(range=['#d485c8', '#39ac9f']).legend(None),
)
diff_chart

alt.Chart(...)

In [11]:
ews = alt.Chart(diff).mark_line(interpolate='step-after').encode(
    alt.X('Datum:T').title(None).axis(grid=False, format="%y", values=['2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01', '2024-01-01']),
    alt.Y('effektive:Q').title(None).axis(values=[0,5,10,15,20,25,30,35,40]),
    color=alt.value("#006d64")
)
ews

alt.Chart(...)

In [12]:
def chart(width):
    return (diff_chart + markt + ews).properties(width=width, height=300+width*0.2, autosize={'type': 'fit', 'contains': 'padding'}).configure_axis(
    labelFont="GT America",
    labelFontSize=12,
    titleFont="GT America",
    titleFontSize=14,
    titleFontWeight=400,
    gridColor="#d4d6dd",
    labelColor="#6e6e7e",
    ticks=False,
    domain=False,
    labelPadding=5
).configure_text(
    fontSize=14,
    font="GT America"
).configure_axisX(
    ticks=True
).configure_view(strokeWidth=0
)
chart(370).save('mw.svg')
chart(370)

alt.LayerChart(...)

In [13]:
chart(630).save('cw.png', scale_factor=4)
chart(630)

alt.LayerChart(...)

Durchschnittliche effektive Vergütung während der Energiekrise (Q4-2021 bis Q4-2022):

In [14]:
d4

,year,effektive
0,2017,8.702552
1,2018,8.120404
2,2019,8.618476
3,2020,8.946350
4,2021,9.594124
5,2022,12.741219
6,2023,14.653950


In [15]:
(d4.query('year == 2022').effektive.iloc[0] * 4 + d4.query('year == 2021').effektive.iloc[0])/5

12.11180027281333